# API - Verification

This notebook illustrates the main features of pyNever for verifying a specification on a NN

In [ ]:
import torch

# Let us reconsider the 2D neural network from the previous example
from pynever import networks, nodes
from pynever.strategies.verification import properties

w = torch.Tensor([[1, 1], [-1, 1]])
b = torch.zeros(2)

nn = networks.SequentialNetwork('nn', 'x')
nn.append_node(nodes.FullyConnectedNode('fc', (2,), 2, weight=w, bias=b))
nn.append_node(nodes.ReLUNode('relu', (2,)))

# And the property
in_coef_mat = torch.Tensor([[1, 0], [-1, 0], [0, 1], [0, -1]])
in_bias_mat = torch.Tensor([1, 1, 1, 1]).unsqueeze(1)
out_coef_mat = [torch.Tensor([[-1, 0]])]
out_bias_mat = [torch.Tensor([-2]).unsqueeze(1)]

my_prop = properties.NeverProperty(in_coef_mat, in_bias_mat, out_coef_mat, out_bias_mat)

## The _algorithms_ module

The module _algorithms_ contains the classes to instantiate the verification problem. The main classes are __VerificationStrategy__ and __VerificationParameters__.

pyNeVer provides two verification algorithms: __SSLP__ and __SSBP__.

- SSLP (Star Sets with Linear Programming) is the first algorithm that we employed for complete and incomplete verification of feed-forward ReLU NNs. The algorithm is described in [this paper](https://link.springer.com/article/10.1007/s00500-024-09907-5)
- SSBP (Star Sets with Bounds Propagation) enhances SSLP with an abstraction-refinement search and symbolic interval propagation. This is the algorithm used in VNNCOMP 2024.

Each algorithm has corresponding parameters to run. The classes __SSLPVerification__ and __SSBPVerification__ will require their __SSLPVerificationParameters__ and __SSBPVerificationParameters__, respectively.

In [ ]:
from pynever.strategies.verification.algorithms import SSLPVerification, SSLPVerificationParameters, SSBPVerification, SSBPVerificationParameters

# SSLPVerification allows 3 heuristics:
# complete - all unstable neurons are split and processed (default)
# overapprox - all unstable neurons are approximated
# mixed - some unstable neurons are split and processed. In this case, neurons_to_refine
# specifies how many neurons in each layer are to process.
params = SSLPVerificationParameters(heuristic='complete', neurons_to_refine=None)
strategy = SSLPVerification(params)

# Let's intercept the logger
import logging
import sys

logger = logging.getLogger('pynever')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

# To run verification, just all the verify() method
strategy.verify(nn, my_prop)

In this case the property is not verified because the output reaches the extreme x0 = 2. If we shift the property a little...

In [ ]:
out_bias_mat = [torch.Tensor([-2.1]).unsqueeze(1)]

my_prop = properties.NeverProperty(in_coef_mat, in_bias_mat, out_coef_mat, out_bias_mat)

strategy.verify(nn, my_prop)

The SSBP algorithm gives the same results, but is generally faster.

SSBPVerification parameters can be tuned to improve the performance on bigger networks:

- heuristic: defines the refinement strategy in the search algorithm. "SEQUENTIAL" refinement refines each neuron in the order they appear in the network; "LOWEST_APPROX" selects the neuron that contributes with the lowest approximation; "LOWEST_APPROX_CURRENT_LAYER" selects the neuron that contributes with the lowest approximation in the layer; "INPUT_BOUNDS_CHANGE" selects the neuron that contributes most to change the input bounds when refined.
- bounds: defines the way bounds are computed (only symbolic interval propagation is used now)
- bounds_direction: defines how to compute the bounds. Forward bounds are faster, but backward bounds are more precise with deep networks.
- intersection: defines the way the output intersection is computed with increasing complexity and precision. "STAR_LP" uses a full LP to check the intersection; "ADAPTIVE" uses different checks based on the state of the problem.
- timeout: a timeout in seconds for the search algorithm

In [ ]:
# Default values for parameters are: heuristic=SEQUENTIAL, bounds=SYMBOLIC,
# bounds_direction=FORWARDS, intersection=ADAPTIVE, timeout=300
strategy = SSBPVerification(SSBPVerificationParameters())
strategy.verify(nn, my_prop)